<a href="https://colab.research.google.com/github/meizhong986/WhisperJAV/blob/main/notebook/WhisperJAV_colab_parallel_expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/meizhong986/WhisperJAV/blob/main/notebook/WhisperJAV_colab_parallel_expert.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/></a>

# WhisperJAV Ensemble Dual-GPU Edition (v1.7.5)

**Universal** - Auto-detects Kaggle T4x2 for parallel processing, falls back to single-GPU on Colab

| Platform | Logic | Storage |
|----------|-------|---------|
| **Kaggle** | **Parallel** (2x T4) | **Input**: `/kaggle/input` (Dataset) <br> **Output**: `/kaggle/working` (Artifacts) |
| **Colab** | **Sequential** (1x GPU) | **Input/Output**: Google Drive |

| Option | What it controls |
|--------|------------------|
| **Scene Detection** | How to split audio into chunks (auditok, silero, semantic) |
| **Speech Segmenter** | How to detect speech in audio (silero, ten) |
| **Speech Enhancer** | Audio cleanup for noisy sources (ffmpeg-dsp, clearvoice, etc.) |
| **Model** | Which AI model to use (large-v2, large-v3, turbo, kotoba) |

---
<div style="font-size: 11px; line-height: 1.4;">
1. <b>Setup Files:</b><br>
&nbsp;&nbsp;&nbsp;• <b>Colab:</b> Upload videos to <code>Google Drive/WhisperJAV/</code><br>
&nbsp;&nbsp;&nbsp;• <b>Kaggle:</b> Add your videos as a Dataset (recommended) or upload to Input.<br>
2. Run <b>Step 1: Expert Configuration</b> (required)<br>
3. Run <b>Step 2: Two-Pass Transcribe</b> (auto-detects platform)<br>
4. Run <b>Step 3: AI Translation</b> (supports Kaggle Secrets/Env/Keys)
</div>

<small>The notebook will automatically disconnect (Colab) or finish session (Kaggle) when done.</small>


In [ ]:
#@title Step 1: Expert Configuration { display-mode: "form" }

#@markdown ## 📁 Files & Output
folder_name = "WhisperJAV" #@param {type:"string"}
subtitle_language = "Japanese" #@param ["Japanese", "English (auto-translate)", "English (AI translate)"]
spoken_language = "Japanese" #@param ["Japanese", "Chinese", "English", "Korean"]

#@markdown ---
#@markdown ## 1️⃣ Pass 1 Configuration (GPU 0)
pass1_quality = "balanced" #@param ["faster", "fast", "balanced", "fidelity", "transformers"]
pass1_sensitivity = "aggressive" #@param ["conservative", "balanced", "aggressive"]
pass1_model = "large-v2" #@param ["automatic", "large-v2", "large-v3", "turbo", "kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"]

#@markdown **Expert Audio Setup (Pass 1)**
pass1_scene_detector = "semantic" #@param ["automatic", "auditok", "silero", "semantic"]
pass1_speech_segmenter = "ten" #@param ["automatic", "silero", "ten", "none"]
pass1_speech_enhancer = "none" #@param ["none", "ffmpeg-dsp", "clearvoice", "zipenhancer", "bs-roformer"]
#@markdown <font size="1">auditok=energy (fast), silero=VAD, semantic=texture (complex audio) | enhancer: ffmpeg-dsp(no GPU), clearvoice(48k), bs-roformer(vocal)</font>

#@markdown **FFmpeg Filters (Pass 1)** *(only if enhancer is ffmpeg-dsp)*
pass1_ffmpeg_amplify = True #@param {type:"boolean"}
pass1_ffmpeg_loudnorm = False #@param {type:"boolean"}
pass1_ffmpeg_compress = False #@param {type:"boolean"}
pass1_ffmpeg_highpass = False #@param {type:"boolean"}

#@markdown ---
#@markdown ## 2️⃣ Pass 2 Configuration (GPU 1)
pass2_quality = "fidelity" #@param ["faster", "fast", "balanced", "fidelity", "transformers"]
pass2_sensitivity = "balanced" #@param ["conservative", "balanced", "aggressive"]
pass2_model = "turbo" #@param ["automatic", "large-v2", "large-v3", "turbo", "kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"]

#@markdown **Expert Audio Setup (Pass 2)**
pass2_scene_detector = "semantic" #@param ["automatic", "auditok", "silero", "semantic"]
pass2_speech_segmenter = "ten" #@param ["automatic", "silero", "ten", "none"]
pass2_speech_enhancer = "ffmpeg-dsp" #@param ["none", "ffmpeg-dsp", "clearvoice", "zipenhancer", "bs-roformer"]

#@markdown **FFmpeg Filters (Pass 2)** *(only if enhancer is ffmpeg-dsp)*
pass2_ffmpeg_amplify = True #@param {type:"boolean"}
pass2_ffmpeg_loudnorm = False #@param {type:"boolean"}
pass2_ffmpeg_compress = False #@param {type:"boolean"}
pass2_ffmpeg_highpass = False #@param {type:"boolean"}

#@markdown ---
#@markdown ## 🔗 Merge Strategy
merge_method = "prefer first pass" #@param ["automatic", "keep all", "prefer first pass", "prefer second pass"]

#@markdown ---
#@markdown ## 🤖 AI Translation *(if selected)*
translation_service = "deepseek" #@param ["deepseek", "openrouter", "gemini", "claude", "gpt"]
api_key = "" #@param {type:"string"}
translation_style = "standard" #@param ["standard", "explicit"]

#@markdown ---
#@markdown ## ⚙️ Session
opening_credit = "" #@param {type:"string"}
closing_credit = "Subs by WhisperJAV" #@param {type:"string"}
auto_disconnect = True #@param {type:"boolean"}

# ═══════════════════════════════════════════
# CONFIGURATION LOGIC
# ═══════════════════════════════════════════

# Mapping dictionaries
combine_map = {"automatic": "smart_merge", "keep all": "full_merge",
               "prefer first pass": "pass1_primary", "prefer second pass": "pass2_primary"}
language_map = {"Japanese": "native", "English (auto-translate)": "direct-to-english",
                "English (AI translate)": "llm"}
tone_map = {"standard": "standard", "explicit": "pornify"}
spoken_language_map = {"Japanese": "japanese", "Chinese": "chinese", "English": "english", "Korean": "korean"}

# Model mapping (None = use pipeline default)
model_map = {
    "automatic": None,
    "large-v2": "large-v2",
    "large-v3": "large-v3",
    "turbo": "turbo",
    "kotoba-bilingual": "kotoba-tech/kotoba-whisper-bilingual-v1.0",
    "kotoba-v2.0": "kotoba-tech/kotoba-whisper-v2.0",
    "kotoba-v2.1": "kotoba-tech/kotoba-whisper-v2.1",
    "kotoba-v2.2": "kotoba-tech/kotoba-whisper-v2.2"
}

# Define model compatibility:
KOTOBA_MODELS = {"kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"}
LEGACY_PIPELINES = {"faster", "fast", "balanced", "fidelity"}

# Auto-correct incompatible model-pipeline combinations
warnings_list = []

# Check Pass 1 compatibility
if pass1_model in KOTOBA_MODELS and pass1_quality in LEGACY_PIPELINES:
    warnings_list.append(f"Pass 1: {pass1_model} requires 'transformers' pipeline. Auto-correcting from '{pass1_quality}' to 'transformers'.")
    pass1_quality = "transformers"

# Check Pass 2 compatibility
if pass2_model in KOTOBA_MODELS and pass2_quality in LEGACY_PIPELINES:
    warnings_list.append(f"Pass 2: {pass2_model} requires 'transformers' pipeline. Auto-correcting from '{pass2_quality}' to 'transformers'.")
    pass2_quality = "transformers"

# Memory warning
heavy_enhancers = {'clearvoice', 'bs-roformer', 'zipenhancer'}
if pass1_speech_enhancer in heavy_enhancers and pass2_speech_enhancer in heavy_enhancers:
    warnings_list.append("Using GPU-based enhancement on both passes may cause OOM on T4 GPU (Sequential Mode). Suggest using ffmpeg-dsp for one pass.")

# Helpers
def build_ffmpeg_filters(amplify, loudnorm, compress, highpass):
    """Combine selected FFmpeg filters into comma-separated string."""
    filters = []
    if amplify: filters.append("amplify")
    if loudnorm: filters.append("loudnorm")
    if compress: filters.append("compress")
    if highpass: filters.append("highpass")
    return ",".join(filters) if filters else None

def map_value(val):
    return None if val == "automatic" else val

def map_segmenter(val):
    return "none" if val == "none" else map_value(val)

# Unified Config Construction
WHISPERJAV_CONFIG = {
    'pass1_pipeline': pass1_quality,
    'pass1_sensitivity': pass1_sensitivity,
    'pass1_speech_segmenter': map_segmenter(pass1_speech_segmenter),
    'pass1_model': model_map[pass1_model],
    'pass2_pipeline': pass2_quality,
    'pass2_sensitivity': pass2_sensitivity,
    'pass2_speech_segmenter': map_segmenter(pass2_speech_segmenter),
    'pass2_model': model_map[pass2_model],
    'merge_strategy': combine_map[merge_method],
    'folder_name': folder_name,
    'subtitle_language': language_map[subtitle_language],
    'language': spoken_language_map[spoken_language],
    'translation_service': translation_service,
    'api_key': api_key,
    'translation_style': tone_map[translation_style],
    'opening_credit': opening_credit,
    'closing_credit': closing_credit,
    'auto_disconnect': auto_disconnect,
    # Compatibility checks for Step 2
    '_pass1_quality': pass1_quality,
    '_pass1_sensitivity': pass1_sensitivity,
    '_pass1_speech_segmenter': pass1_speech_segmenter,
    '_pass1_model': pass1_model,
    '_pass2_quality': pass2_quality,
    '_pass2_sensitivity': pass2_sensitivity,
    '_pass2_speech_segmenter': pass2_speech_segmenter,
    '_pass2_model': pass2_model,
    '_merge_method': merge_method,
    '_subtitle_language': subtitle_language,
    '_translation_style': translation_style,
}

WHISPERJAV_EXPERT_CONFIG = {
    # Pass 1 Expert
    'pass1_scene_detector': map_value(pass1_scene_detector),
    'pass1_speech_segmenter': map_segmenter(pass1_speech_segmenter), # Now unified
    'pass1_speech_enhancer': None if pass1_speech_enhancer == "none" else pass1_speech_enhancer,
    'pass1_ffmpeg_filters': build_ffmpeg_filters(pass1_ffmpeg_amplify, pass1_ffmpeg_loudnorm, pass1_ffmpeg_compress, pass1_ffmpeg_highpass) if pass1_speech_enhancer == "ffmpeg-dsp" else None,
    # Pass 2 Expert
    'pass2_scene_detector': map_value(pass2_scene_detector),
    'pass2_speech_segmenter': map_segmenter(pass2_speech_segmenter), # Now unified
    'pass2_speech_enhancer': None if pass2_speech_enhancer == "none" else pass2_speech_enhancer,
    'pass2_ffmpeg_filters': build_ffmpeg_filters(pass2_ffmpeg_amplify, pass2_ffmpeg_loudnorm, pass2_ffmpeg_compress, pass2_ffmpeg_highpass) if pass2_speech_enhancer == "ffmpeg-dsp" else None,
    # Display helpers
    '_pass1_scene_detector': pass1_scene_detector,
    '_pass1_speech_enhancer': pass1_speech_enhancer,
    '_pass2_scene_detector': pass2_scene_detector,
    '_pass2_speech_enhancer': pass2_speech_enhancer,
}

from IPython.display import display, HTML

# Display warnings
for warning in warnings_list:
    display(HTML(f'<div style="padding:6px 10px;background:#fef9c3;border-radius:4px;font-size:10px;margin-bottom:4px"><b>⚠️ Auto-corrected:</b> {warning}</div>'))

# Build status display
p1_info = f"{pass1_quality}"
if pass1_speech_segmenter != "automatic":
    p1_info += f"/{pass1_speech_segmenter}"
if pass1_model != "automatic":
    p1_info += f"/{pass1_model}"

p2_info = f"{pass2_quality}"
if pass2_speech_segmenter != "automatic":
    p2_info += f"/{pass2_speech_segmenter}"
if pass2_model != "automatic":
    p2_info += f"/{pass2_model}"

display(HTML(f'<div style="padding:10px;background:#e0f2fe;border-radius:4px;font-size:11px">'
             f'<b>Parallel Configuration Loaded</b><br>'
             f'Pass 1: {p1_info} | Pass 2: {p2_info}<br>'
             f'Merge: {merge_method} | Folder: {folder_name} | Language: {spoken_language}'
             f'</div>'))

In [ ]:
#@title Step 2: Two-Pass Transcribe { display-mode: "form" }
#@markdown Connect Drive (Colab) or Setup Paths (Kaggle) → Install → Run passes → Merge results

import os, sys, subprocess, shlex, time, re, shutil
from pathlib import Path
from IPython.display import display, clear_output
from IPython.core.display import HTML
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple, Optional

def status(msg, ok=True):
    icon = "✓" if ok else "✗"
    print(f"{icon} {msg}")

def section(title):
    print(f"\n{'─'*50}\n{title}\n{'─'*50}")


def tail_lines(path: Optional[Path], n: int = 30, max_chars: int = 4000) -> str:
    """Return the last N lines of a text file (best-effort) for notebook status panels."""
    if not path or not path.exists():
        return ""
    try:
        text = path.read_text(encoding="utf-8", errors="replace")
    except Exception:
        try:
            text = path.read_text(encoding="utf-8")
        except Exception:
            return ""
    lines = text.splitlines()
    tail = "\n".join(lines[-n:])
    if max_chars and len(tail) > max_chars:
        tail = tail[-max_chars:]
    return tail


def render_pass_panel(pass_states: Dict[int, Dict[str, Any]], tail_n: int = 4) -> None:
    """Render a compact, overwrite-in-place status panel for Kaggle/Colab notebooks."""
    clear_output(wait=True)
    section("EXECUTION (LIVE STATUS)")

    for pass_num in sorted(pass_states.keys()):
        st = pass_states[pass_num]
        status_txt = st.get("status", "?")
        elapsed = st.get("elapsed", 0.0)
        rc = st.get("returncode")
        log_path = st.get("log_path")
        extra = ""
        if rc is not None:
            extra = f" | rc={rc}"
        print(f"Pass {pass_num}: {status_txt} | {elapsed:.0f}s{extra}")
        if log_path:
            tail = tail_lines(Path(log_path), n=tail_n)
            if tail:
                print("--- log tail ---")
                print(tail)
        print()

# Check config
if 'WHISPERJAV_CONFIG' not in dir():
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> Run Step 1 first</div>'))
    raise SystemExit()
cfg = WHISPERJAV_CONFIG

# Check for expert config (always present now)
expert = WHISPERJAV_EXPERT_CONFIG if 'WHISPERJAV_EXPERT_CONFIG' in dir() else None

# ═══════════════════════════════════════════
# PLATFORM DETECTION & FILE SYSTEM SETUP
# ═══════════════════════════════════════════
section("DETECTING PLATFORM & PATHS")

def detect_platform():
    if 'google.colab' in sys.modules:
        return 'colab'
    elif os.path.exists('/kaggle'):
        return 'kaggle'
    else:
        return 'local'

PLATFORM = detect_platform()
print(f"Platform: {PLATFORM.upper()}")

if PLATFORM == 'colab':
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        base_dir = Path(f"/content/drive/MyDrive")
        input_dir = base_dir / cfg['folder_name']
        output_dir = input_dir # Colab writes back to same folder
        input_dir.mkdir(parents=True, exist_ok=True)
        status(f"Drive Connected: {input_dir}")
    except Exception as e:
        status(f"Failed to connect Drive: {e}", False)
        raise SystemExit()

elif PLATFORM == 'kaggle':
    # Kaggle: Read-only Input, specific Output
    # Search for input dataset matching folder_name or standard locations
    potential_inputs = [
        Path(f"/kaggle/input/{cfg['folder_name']}"), # Named dataset
        Path(f"/kaggle/input/whisperjav-media"),    # Common dataset name
        Path(f"/kaggle/input"),                     # Root input
        Path(f"/kaggle/working/input")              # Manual upload location
    ]
    
    input_dir = None
    for p in potential_inputs:
        if p.exists() and any(f for f in p.rglob('*') if f.suffix.lower() in {'.mp4', '.mkv', '.avi', '.mp3', '.wav', '.flac'}):
            input_dir = p
            break
            
    if not input_dir:
        input_dir = Path(f"/kaggle/input")
        print("  ⚠ Could not auto-detect specific input folder. Scanning /kaggle/input root.")

    output_dir = Path(f"/kaggle/working/{cfg['folder_name']}")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # KAGGLE FIX 1: Fix Temp Storage Overflow
    # Redirect all temp files to writable working dir to avoid /tmp (ramdisk) filling up
    kaggle_temp = Path(f"/kaggle/working/temp")
    kaggle_temp.mkdir(parents=True, exist_ok=True)
    os.environ['TMPDIR'] = str(kaggle_temp)
    os.environ['TEMP'] = str(kaggle_temp)
    os.environ['TMP'] = str(kaggle_temp)
    print(f"  🔧 Redirected temp storage to {kaggle_temp}")

    # KAGGLE CHECK 2: Internet Access
    try:
        import socket
        socket.create_connection(("www.google.com", 80), timeout=2)
    except OSError:
        display(HTML('<div style="padding:10px;background:#fee2e2;border-radius:4px;border-left:4px solid #ef4444;color:#991b1b;"><b>🛑 Internet Disabled!</b><br>You must enable "Internet" in the Notebook Settings (right sidebar) > Internet > On.<br>Otherwise installation will fail.</div>'))
        time.sleep(5)

    status(f"Input: {input_dir}")
    status(f"Output: {output_dir}")

else:
    # Local fallback
    input_dir = Path(cfg['folder_name'])
    output_dir = Path(f"{cfg['folder_name']}_output")
    output_dir.mkdir(parents=True, exist_ok=True)
    status(f"Local Mode: {input_dir} → {output_dir}")

# ═══════════════════════════════════════════
# CHECK GPUs AND DETERMINE MODE
# ═══════════════════════════════════════════
section("GPU RESOURCE CHECK")

# Fail-fast: this notebook is intended for GPU runtimes (Kaggle/Colab).
gpu_check = subprocess.run(
    "nvidia-smi --query-gpu=name,memory.total --format=csv,noheader",
    shell=True,
    capture_output=True,
    text=True,
 )

if gpu_check.returncode != 0 or not gpu_check.stdout.strip():
    status("No GPU detected. Switch runtime to GPU (T4 x2 or L4/A100).", False)
    if PLATFORM == 'kaggle':
        print("  Kaggle: right sidebar → Settings → Accelerator → GPU (T4 x2) → Save.")
    elif PLATFORM == 'colab':
        print("  Colab: Runtime → Change runtime type → Hardware accelerator: GPU.")
    else:
        print("  Local: ensure NVIDIA drivers are installed and `nvidia-smi` works.")
    raise SystemExit()

gpu_lines = [line.strip() for line in gpu_check.stdout.strip().split('\n') if line.strip()]
num_gpus = len(gpu_lines)

for i, gpu_info in enumerate(gpu_lines):
    status(f"GPU {i}: {gpu_info}")

# Adaptive mode selection
if num_gpus >= 2:
    PARALLEL_MODE = True
    gpu_assignment = {1: "0", 2: "1"}
    print(f"\n  ⚡ Parallel Mode: Pass 1 (GPU 0) | Pass 2 (GPU 1)")
else:
    PARALLEL_MODE = False
    gpu_assignment = {1: "0", 2: "0"}
    print(f"\n  📝 Sequential Mode: Pass 1 → Pass 2 (Single GPU)")

# ═══════════════════════════════════════════
# INSTALL WHISPERJAV
# ═══════════════════════════════════════════
section("INSTALLING (2-3 min)")
install_start = time.time()

# Platform specific prereqs
if PLATFORM == 'colab':
    pre_cmds = "apt-get update -qq && apt-get install -y -qq ffmpeg portaudio19-dev libc++1 libc++abi1 > /dev/null 2>&1"
else:
    # Kaggle usually has ffmpeg, but good to ensure
    pre_cmds = "apt-get update -qq && apt-get install -y -qq ffmpeg portaudio19-dev > /dev/null 2>&1"

# ARCHITECTURE NOTE:
# WhisperJAV requires NumPy>=2.0. Standard Kaggle/Colab environments use NumPy 1.x.
# To satisfy the requirement without breaking the ecosystem (matplotlib/torch ABI mismatches),
# we must perform a synchronized upgrade of the core stack (numpy, matplotlib, torch).
# We rely on 'configure_cuda_runtime_paths()' (below) to fix any CUDA version mismatches 
# caused by replacing the system PyTorch.

req_packages = [
    # Core Audio/Video
    "ffmpeg-python", "auditok", "pysrt", "srt", "aiofiles", "pyloudnorm", "pydub",
    # AI/ML Stack (Synchronized Upgrade)
    "numpy", "matplotlib", "scipy", "torch", "torchaudio", "torchvision",
    "faster-whisper", "transformers", "optimum", "accelerate", "huggingface-hub", 
    "pydantic", "ten-vad", "silero-vad", "modelscope", "addict", "tiktoken"
 ]
req_str = " ".join(req_packages)

steps = [
    (pre_cmds, "System tools"),
    # Install dependencies allowing upgrades to ensure consistency (Numpy 2.0 compatible stack)
    (f"pip install -q {req_str}", "Python packages (Core Stack)"),
    ("pip install -q --no-deps git+https://github.com/openai/whisper.git@main", "Whisper"),
    ("pip install -q --no-deps git+https://github.com/meizhong986/stable-ts-fix-setup.git@main", "Stable-TS"),
    ("pip install -q --no-deps git+https://github.com/meizhong986/WhisperJAV.git@main", "WhisperJAV")
]

for cmd, name in steps:
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        status(f"{name} failed", False)
        # On Kaggle, some apt-get might fail but environment is often usable
        if PLATFORM != 'kaggle': 
            raise SystemExit()
    else:
        status(name)

# Conditional installation of speech enhancer dependencies
if expert:
    extra_packages = set()
    for enhancer in [expert.get('pass1_speech_enhancer'), expert.get('pass2_speech_enhancer')]:
        if enhancer == 'clearvoice':
            extra_packages.add('clearvoice')
        elif enhancer == 'zipenhancer':
            # zipenhancer uses modelscope which is already installed above
            pass
        elif enhancer == 'bs-roformer':
            extra_packages.add('bs-roformer-infer')
    
    if extra_packages:
        pkg_list = ' '.join(extra_packages)
        result = subprocess.run(f"pip install -q {pkg_list}", shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            status(f"Speech enhancer packages failed (continuing anyway)", False)
        else:
            status(f"Speech enhancer packages ({', '.join(extra_packages)})")

status(f"Installation complete ({time.time()-install_start:.0f}s)")

# ═══════════════════════════════════════════
# RUNTIME CONFIGURATION (Fixing CUDA conflicts)
# ═══════════════════════════════════════════
def configure_cuda_runtime_paths():
    """
    Kaggle/Colab Fix: Link pip-installed NVIDIA libraries to LD_LIBRARY_PATH.
    This prevents ctranslate2 (faster-whisper) from using incompatible system CUDA 11.8.
    """
    try:
        import site
        nvidia_paths = []
        for site_pkg in site.getsitepackages():
            nvidia_root = Path(site_pkg) / 'nvidia'
            if nvidia_root.exists():
                nvidia_paths.extend([str(p / 'lib') for p in nvidia_root.iterdir() if (p / 'lib').exists()])
        
        if nvidia_paths:
            current_ld = os.environ.get('LD_LIBRARY_PATH', '')
            new_ld = os.pathsep.join(nvidia_paths + [current_ld] if current_ld else nvidia_paths)
            os.environ['LD_LIBRARY_PATH'] = new_ld
            print(f"  🔧 Runtime: Linked {len(nvidia_paths)} NVIDIA library paths for ctranslate2")
    except Exception as e:
        print(f"  ⚠ Runtime Warning: {e}")

configure_cuda_runtime_paths()

# ═══════════════════════════════════════════
# FIND MEDIA FILES
# ═══════════════════════════════════════════
section("SCANNING INPUT FILES")
video_types = {'.mp4', '.mkv', '.avi', '.mov', '.wmv', '.flv', '.webm', '.m4v', '.mp3', '.wav', '.flac', '.m4a'}

# Recursive scan for Kaggle datasets structure
if PLATFORM == 'kaggle':
    videos = []
    for f in input_dir.rglob('*'):
        if f.suffix.lower() in video_types:
            videos.append(f)
else:
    videos = [f for f in input_dir.iterdir() if f.suffix.lower() in video_types]

if not videos:
    status(f"No media files found in {input_dir}", False)
    if PLATFORM == 'kaggle':
         print("  Hint: Did you add your dataset via '+ Add Data'?")
    raise SystemExit()

status(f"Found {len(videos)} file(s)")
for v in videos[:5]:
    print(f"  • {v.name}")
if len(videos) > 5:
    print(f"  ... and {len(videos)-5} more")

# ═══════════════════════════════════════════
# MERGE FUNCTIONS (from whisperjav/ensemble/merge.py)
# ═══════════════════════════════════════════

# Prefer WhisperJAV's own merge engine for correctness.

_merge_srt_impl = None
try:
    from whisperjav.ensemble.merge import MergeEngine
    _wj_merger_obj = MergeEngine()

    def _merge_srt_impl(
        srt1: Path,
        srt2: Path,
        output: Path,
        strategy: str,
        _merger: Any = _wj_merger_obj,
    ) -> Dict[str, Any]:
        return _merger.merge(srt1, srt2, output, strategy=strategy)
except Exception as e:
    print(f"  ⚠ WhisperJAV MergeEngine unavailable, using notebook fallback merge: {e}")

merge_srt = _merge_srt_impl

@dataclass
class Subtitle:
    index: int
    start_time: float
    end_time: float
    text: str

    @property
    def duration(self) -> float:
        return self.end_time - self.start_time

def parse_srt(path: Path) -> List[Subtitle]:
    if not path.exists():
        return []
    subtitles = []
    content = path.read_text(encoding='utf-8-sig')
    blocks = re.split(r'\n\s*\n', content.strip())
    for block in blocks:
        if not block.strip():
            continue
        lines = block.strip().split('\n')
        if len(lines) < 3:
            continue
        try:
            index = int(lines[0].strip())
            ts_match = re.match(r'(\d{2}):(\d{2}):(\d{2}),(\d{3})\s*-->\s*(\d{2}):(\d{2}):(\d{2}),(\d{3})', lines[1].strip())
            if not ts_match:
                continue
            g = ts_match.groups()
            start = int(g[0])*3600 + int(g[1])*60 + int(g[2]) + int(g[3])/1000
            end = int(g[4])*3600 + int(g[5])*60 + int(g[6]) + int(g[7])/1000
            text = '\n'.join(lines[2:]).strip()
            subtitles.append(Subtitle(index, start, end, text))
        except:
            continue
    return subtitles

def write_srt(subtitles: List[Subtitle], path: Path):
    def ts(seconds):
        h, m = int(seconds // 3600), int((seconds % 3600) // 60)
        s, ms = int(seconds % 60), int((seconds % 1) * 1000)
        return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"
    lines = []
    for i, sub in enumerate(subtitles, 1):
        lines.extend([str(i), f"{ts(sub.start_time)} --> {ts(sub.end_time)}", sub.text, ''])
    path.write_text('\n'.join(lines), encoding='utf-8')

def preview_srt(path: Optional[Path], label: str, n: int = 3):
    """Beta-debug helper: print first/last few lines of an SRT for easy field reports."""
    if not path:
        print(f"        (preview) {label}: <none>")
        return
    p = Path(path)
    if not p.exists():
        print(f"        (preview) {label}: missing ({p})")
        return
    try:
        text = p.read_text(encoding='utf-8-sig', errors='replace')
    except TypeError:
        text = p.read_text(encoding='utf-8-sig')
    lines = text.splitlines()
    if not lines:
        print(f"        (preview) {label}: empty")
        return
    head = lines[:n]
    tail = lines[-n:] if len(lines) > n else lines
    print(f"        (preview) {label}: head")
    for ln in head:
        print(f"          {ln}")
    if len(lines) > n:
        print(f"        (preview) {label}: tail")
        for ln in tail:
            print(f"          {ln}")

if _merge_srt_impl is None:
    def _merge_srt_impl(srt1: Path, srt2: Path, output: Path, strategy: str) -> Dict[str, Any]:
        subs1, subs2 = parse_srt(srt1), parse_srt(srt2)
        
        if strategy == 'full_merge':
            merged = [Subtitle(0, s.start_time, s.end_time, s.text) for s in subs1 + subs2]
        elif strategy == 'pass1_primary':
            merged = [Subtitle(0, s.start_time, s.end_time, s.text) for s in subs1]
            for s2 in subs2:
                if not any(max(s1.start_time, s2.start_time) < min(s1.end_time, s2.end_time) for s1 in subs1):
                    merged.append(Subtitle(0, s2.start_time, s2.end_time, s2.text))
        elif strategy == 'pass2_primary':
            merged = [Subtitle(0, s.start_time, s.end_time, s.text) for s in subs2]
            for s1 in subs1:
                if not any(max(s1.start_time, s2.start_time) < min(s1.end_time, s2.end_time) for s2 in subs2):
                    merged.append(Subtitle(0, s1.start_time, s1.end_time, s1.text))
        else:  # smart_merge
            merged, used = [], set()
            for s1 in subs1:
                best_i, best_overlap = None, 0
                for i, s2 in enumerate(subs2):
                    if i in used: continue
                    overlap = max(0, min(s1.end_time, s2.end_time) - max(s1.start_time, s2.start_time))
                    if overlap > best_overlap:
                        best_overlap, best_i = overlap, i
                if best_i is not None and best_overlap > 0.3 * min(s1.duration, subs2[best_i].duration):
                    used.add(best_i)
                    chosen = s1 if s1.duration <= subs2[best_i].duration else subs2[best_i]
                    merged.append(Subtitle(0, chosen.start_time, chosen.end_time, chosen.text))
                else:
                    merged.append(Subtitle(0, s1.start_time, s1.end_time, s1.text))
            for i, s2 in enumerate(subs2):
                if i not in used:
                    merged.append(Subtitle(0, s2.start_time, s2.end_time, s2.text))
        
        merged.sort(key=lambda s: s.start_time)
        write_srt(merged, output)
        return {'pass1_count': len(subs1), 'pass2_count': len(subs2), 'merged_count': len(merged), 'strategy': strategy}

    merge_srt = _merge_srt_impl

# ═══════════════════════════════════════════
# TWO-PASS TRANSCRIPTION
# ═══════════════════════════════════════════
section("EXECUTION")

def build_pass_command(pass_num: int, video_path: Path, work_dir: Path, cfg: dict, expert: Optional[dict] = None) -> Tuple[List[str], Path]:
    """Build whisperjav command for a single pass."""
    
    # Separate directory for each pass to avoid conflicts
    # On Kaggle we must write to /kaggle/working (passed as work_dir)
    pass_output_dir = work_dir / f"pass{pass_num}"
    pass_output_dir.mkdir(parents=True, exist_ok=True)

    # Use unique local temp directory inside the working directory to ensure space
    pass_temp_dir = work_dir / f"temp_pass{pass_num}"
    pass_temp_dir.mkdir(parents=True, exist_ok=True)

    pipeline = cfg[f'pass{pass_num}_pipeline']
    sensitivity = cfg[f'pass{pass_num}_sensitivity']
    segmenter = cfg[f'pass{pass_num}_speech_segmenter']
    model = cfg[f'pass{pass_num}_model']

    cmd = [
        'whisperjav', str(video_path),
        '--output-dir', str(pass_output_dir),
        '--temp-dir', str(pass_temp_dir),
        '--ensemble',
        '--pass1-pipeline', pipeline,
        '--pass1-sensitivity', sensitivity,
    ]
    # Beta debug: extra visibility + artifacts
    if cfg.get('beta_debug', True):
        cmd.extend(['--log-level', 'DEBUG'])
        cmd.extend(['--stats-file', str(pass_output_dir / 'stats.json')])
        cmd.extend(['--trace-params', str(pass_output_dir / 'trace_params.jsonl')])

    if segmenter:
        cmd.extend(['--pass1-speech-segmenter', segmenter])

    if model:
        cmd.extend(['--pass1-model', model])

    if expert:
        scene_detector = expert.get(f'pass{pass_num}_scene_detector')
        if scene_detector:
            cmd.extend(['--pass1-scene-detector', scene_detector])
        
        speech_enhancer = expert.get(f'pass{pass_num}_speech_enhancer')
        if speech_enhancer:
            if speech_enhancer == 'ffmpeg-dsp':
                effects = expert.get(f'pass{pass_num}_ffmpeg_filters')
                effects_str = effects if effects else 'amplify'
                cmd.extend(['--pass1-speech-enhancer', f'ffmpeg-dsp:{effects_str}'])
            else:
                cmd.extend(['--pass1-speech-enhancer', speech_enhancer])

    if cfg['subtitle_language'] == 'direct-to-english':
        cmd.extend(['--subs-language', 'direct-to-english'])
    else:
        cmd.extend(['--subs-language', 'native'])

    lang = cfg.get('language')
    if lang:
        cmd.extend(['--language', str(lang)])

    return cmd, pass_output_dir



def build_twopass_command(video_path: Path, work_dir: Path, cfg: dict, expert: Optional[dict] = None) -> Tuple[List[str], Path]:
    """Build a single whisperjav command that runs Pass 1 + Pass 2 in one invocation (single GPU mode)."""

    run_output_dir = work_dir / f"twopass_{video_path.stem}"
    run_output_dir.mkdir(parents=True, exist_ok=True)

    run_temp_dir = work_dir / f"temp_twopass_{video_path.stem}"
    run_temp_dir.mkdir(parents=True, exist_ok=True)

    cmd: List[str] = [
        'whisperjav', str(video_path),
        '--output-dir', str(run_output_dir),
        '--temp-dir', str(run_temp_dir),
        '--ensemble',
        '--pass1-pipeline', cfg['pass1_pipeline'],
        '--pass1-sensitivity', cfg['pass1_sensitivity'],
        '--pass2-pipeline', cfg['pass2_pipeline'],
        '--pass2-sensitivity', cfg['pass2_sensitivity'],
        '--merge-strategy', cfg['merge_strategy'],
    ]

    if cfg.get('beta_debug', True):
        cmd.extend(['--log-level', 'DEBUG'])
        cmd.extend(['--stats-file', str(run_output_dir / 'stats.json')])
        cmd.extend(['--trace-params', str(run_output_dir / 'trace_params.jsonl')])

    if cfg.get('pass1_speech_segmenter'):
        cmd.extend(['--pass1-speech-segmenter', cfg['pass1_speech_segmenter']])
    if cfg.get('pass2_speech_segmenter'):
        cmd.extend(['--pass2-speech-segmenter', cfg['pass2_speech_segmenter']])

    if cfg.get('pass1_model'):
        cmd.extend(['--pass1-model', cfg['pass1_model']])
    if cfg.get('pass2_model'):
        cmd.extend(['--pass2-model', cfg['pass2_model']])

    if expert:
        p1_scene = expert.get('pass1_scene_detector')
        if p1_scene:
            cmd.extend(['--pass1-scene-detector', p1_scene])
        p2_scene = expert.get('pass2_scene_detector')
        if p2_scene:
            cmd.extend(['--pass2-scene-detector', p2_scene])

        p1_enh = expert.get('pass1_speech_enhancer')
        if p1_enh:
            if p1_enh == 'ffmpeg-dsp':
                effects = expert.get('pass1_ffmpeg_filters')
                effects_str = effects if effects else 'amplify'
                cmd.extend(['--pass1-speech-enhancer', f'ffmpeg-dsp:{effects_str}'])
            else:
                cmd.extend(['--pass1-speech-enhancer', p1_enh])

        p2_enh = expert.get('pass2_speech_enhancer')
        if p2_enh:
            if p2_enh == 'ffmpeg-dsp':
                effects = expert.get('pass2_ffmpeg_filters')
                effects_str = effects if effects else 'amplify'
                cmd.extend(['--pass2-speech-enhancer', f'ffmpeg-dsp:{effects_str}'])
            else:
                cmd.extend(['--pass2-speech-enhancer', p2_enh])

    if cfg.get('subtitle_language') == 'direct-to-english':
        cmd.extend(['--subs-language', 'direct-to-english'])
    else:
        cmd.extend(['--subs-language', 'native'])

    lang = cfg.get('language')
    if lang:
        cmd.extend(['--language', str(lang)])

    return cmd, run_output_dir


def run_twopass(video: Path, work_dir: Path, cfg: dict, expert: Optional[dict], gpu_id: str) -> Dict[str, Any]:
    """Run a single 2-pass whisperjav invocation (single GPU mode)."""

    cmd, run_output_dir = build_twopass_command(video, work_dir, cfg, expert)
    log_path = run_output_dir / 'run.log'

    env = os.environ.copy()
    env['CUDA_VISIBLE_DEVICES'] = gpu_id

    full_cmd = shlex.join(cmd)
    if cfg.get('beta_debug', True):
        print(f"      cmd(twopass): {full_cmd}")
        print(f"      log(twopass): {log_path}")

    log_f = open(log_path, 'w', encoding='utf-8', errors='replace')
    proc = subprocess.Popen(
        full_cmd,
        shell=True,
        stdout=log_f,
        stderr=subprocess.STDOUT,
        text=True,
        env=env,
        bufsize=1,
    )

    ctx = {
        'pass': 0,
        'video': video.name,
        'gpu': gpu_id,
        'proc': proc,
        'log_f': log_f,
        'log_path': log_path,
        'output_dir': run_output_dir,
        'start_time': time.time(),
    }

    monitor_processes({1: ctx}, refresh_sec=8.0)
    proc.wait()

    try:
        log_f.close()
    except Exception:
        pass

    actual_output = find_output_srt(run_output_dir, video.name)
    cue_count = len(parse_srt(actual_output)) if actual_output and actual_output.exists() else 0
    output_bytes = actual_output.stat().st_size if actual_output and actual_output.exists() else 0

    return {
        'pass': 0,
        'video': video.name,
        'success': (proc.returncode == 0 and actual_output and actual_output.exists()),
        'output': actual_output,
        'output_dir': run_output_dir,
        'elapsed': time.time() - ctx['start_time'],
        'gpu': gpu_id,
        'cue_count': cue_count,
        'output_bytes': output_bytes,
        'returncode': proc.returncode,
        'log_path': log_path,
    }


def start_pass(pass_num: int, video: Path, work_dir: Path, cfg: dict, expert: Optional[dict], gpu_id: str) -> Dict[str, Any]:
    """Start a pass as a background process, logging stdout/stderr to a file."""
    cmd, pass_output_dir = build_pass_command(pass_num, video, work_dir, cfg, expert)
    log_path = pass_output_dir / "run.log"
    env = os.environ.copy()
    env['CUDA_VISIBLE_DEVICES'] = gpu_id

    full_cmd = shlex.join(cmd)
    if cfg.get('beta_debug', True):
        print(f"      cmd(pass{pass_num}): {full_cmd}")
        print(f"      log(pass{pass_num}): {log_path}")

    log_f = open(log_path, "w", encoding="utf-8", errors="replace")
    proc = subprocess.Popen(
        full_cmd,
        shell=True,
        stdout=log_f,
        stderr=subprocess.STDOUT,
        text=True,
        env=env,
        bufsize=1,
    )

    return {
        "pass": pass_num,
        "video": video.name,
        "gpu": gpu_id,
        "proc": proc,
        "log_f": log_f,
        "log_path": log_path,
        "output_dir": pass_output_dir,
        "start_time": time.time(),
    }


def finalize_pass(pass_ctx: Dict[str, Any], video: Path) -> Dict[str, Any]:
    """Collect results after a pass finishes."""
    proc = pass_ctx["proc"]
    pass_output_dir = pass_ctx["output_dir"]
    log_path = pass_ctx["log_path"]
    start_time = pass_ctx["start_time"]
    elapsed = time.time() - start_time

    try:
        pass_ctx["log_f"].close()
    except Exception:
        pass

    actual_output = find_output_srt(pass_output_dir, video.name)
    cue_count = len(parse_srt(actual_output)) if actual_output and actual_output.exists() else 0
    output_bytes = actual_output.stat().st_size if actual_output and actual_output.exists() else 0

    return {
        'pass': pass_ctx['pass'],
        'video': video.name,
        'success': (proc.returncode == 0 and actual_output and actual_output.exists()),
        'output': actual_output,
        'output_dir': pass_output_dir,
        'elapsed': elapsed,
        'gpu': pass_ctx['gpu'],
        'cue_count': cue_count,
        'output_bytes': output_bytes,
        'returncode': proc.returncode,
        'log_path': log_path,
    }


def monitor_processes(pass_ctxs: Dict[int, Dict[str, Any]], refresh_sec: float = 8.0) -> None:
    """Poll running processes and show a live notebook status panel."""
    pass_states: Dict[int, Dict[str, Any]] = {}
    for pnum, ctx in pass_ctxs.items():
        pass_states[pnum] = {
            "status": "running",
            "elapsed": 0.0,
            "returncode": None,
            "log_path": ctx.get("log_path"),
        }

    while True:
        any_running = False
        for pnum, ctx in pass_ctxs.items():
            proc = ctx["proc"]
            rc = proc.poll()
            elapsed = time.time() - ctx["start_time"]
            if rc is None:
                any_running = True
                pass_states[pnum].update({"status": "running", "elapsed": elapsed, "returncode": None})
            else:
                pass_states[pnum].update({"status": "done", "elapsed": elapsed, "returncode": rc})

        render_pass_panel(pass_states, tail_n=4)
        if not any_running:
            break
        time.sleep(refresh_sec)

def find_output_srt(pass_output_dir: Path, video_name: str) -> Optional[Path]:
    """Find the generated SRT file."""
    base_name = Path(video_name).stem
    patterns = [
        f"{base_name}.*.whisperjav.srt",
        f"{base_name}.srt",
        f"{base_name}*.srt",
    ]
    for pattern in patterns:
        matches = list(pass_output_dir.glob(pattern))
        if matches:
            return matches[0]
    all_srts = list(pass_output_dir.glob("*.srt"))
    return all_srts[0] if all_srts else None

def run_pass(pass_num: int, video: Path, work_dir: Path, cfg: dict, expert: Optional[dict], gpu_id: str) -> Dict:
    """Run a single pass on a specific GPU (kept for sequential mode)."""
    pass_ctx = start_pass(pass_num, video, work_dir, cfg, expert, gpu_id)
    monitor_processes({pass_num: pass_ctx}, refresh_sec=8.0)
    pass_ctx['proc'].wait()
    return finalize_pass(pass_ctx, video)

# Info display
p1_info = cfg['_pass1_quality']
if cfg['_pass1_speech_segmenter'] != 'automatic': p1_info += f"/{cfg['_pass1_speech_segmenter']}"
p2_info = cfg['_pass2_quality']
if cfg['_pass2_speech_segmenter'] != 'automatic': p2_info += f"/{cfg['_pass2_speech_segmenter']}"

print(f"Pass 1: {p1_info}")
print(f"Pass 2: {p2_info}")
print(f"Merge: {cfg['_merge_method']}\n")

# Process each video
all_results = []
merged_outputs = []

for video_idx, video in enumerate(videos, 1):
    print(f"\n[{video_idx}/{len(videos)}] Processing: {video.name}")

    results = {}

    if PARALLEL_MODE:
        # Option A: per-pass logs + a live notebook status panel (no interleaved output)
        pass1_ctx = start_pass(1, video, output_dir, cfg, expert, gpu_assignment[1])
        pass2_ctx = start_pass(2, video, output_dir, cfg, expert, gpu_assignment[2])
        monitor_processes({1: pass1_ctx, 2: pass2_ctx}, refresh_sec=8.0)
        pass1_ctx['proc'].wait()
        pass2_ctx['proc'].wait()
        results[1] = finalize_pass(pass1_ctx, video)
        results[2] = finalize_pass(pass2_ctx, video)

        # Print a small completion summary (permanent output)
        for pass_num in [1, 2]:
            result = results[pass_num]
            status_icon = "✓" if result['success'] else "✗"
            print(f"    {status_icon} Pass {pass_num} (GPU {result['gpu']}): {result['elapsed']:.1f}s | cues={result.get('cue_count', 0)} | bytes={result.get('output_bytes', 0)}")
            if cfg.get('beta_debug', True) and result['success']:
                preview_srt(result.get('output'), f"pass{pass_num}", n=3)
            if not result['success']:
                print(f"        log: {result.get('log_path')}")
                tail = tail_lines(result.get('log_path'), n=25)
                if tail:
                    print("        log tail:")
                    print(tail)
    else:
        twopass = run_twopass(video, output_dir, cfg, expert, gpu_assignment[1])
        status_icon = '✓' if twopass['success'] else '✗'
        print(f"    {status_icon} Two-pass (single GPU {twopass['gpu']}): {twopass['elapsed']:.1f}s | cues={twopass.get('cue_count', 0)} | bytes={twopass.get('output_bytes', 0)}")
        if cfg.get('beta_debug', True) and twopass['success']:
            preview_srt(twopass.get('output'), 'twopass', n=3)
        if not twopass['success']:
            print(f"        log: {twopass.get('log_path')}")
            tail = tail_lines(twopass.get('log_path'), n=25)
            if tail:
                print('        log tail:')
                print(tail)

    # Merge or copy
    if not PARALLEL_MODE:
        if twopass.get('success') and twopass.get('output') and Path(twopass['output']).exists():
            final_output = output_dir / f"{video.stem}.whisperjav.srt"
            shutil.copy2(twopass['output'], final_output)
            merged_outputs.append(final_output)
            print('    ✓ Produced final subtitle from two-pass run')
            if cfg.get('beta_debug', True):
                preview_srt(final_output, 'final(twopass)', n=3)
        else:
            print('    ✗ Two-pass run failed!')
        all_results.append({0: twopass})
        continue

    if results[1]['success'] and results[2]['success']:
        merged_output = output_dir / f"{video.stem}.merged.whisperjav.srt"
        stats = merge_srt(results[1]['output'], results[2]['output'], merged_output, cfg['merge_strategy'])
        merged_cues = len(parse_srt(merged_output)) if merged_output.exists() else 0
        merged_bytes = merged_output.stat().st_size if merged_output.exists() else 0
        print(f"    ✓ Merged: {stats['pass1_count']} + {stats['pass2_count']} → {stats.get('merged_count', 0)} subtitles | merged_cues={merged_cues} | bytes={merged_bytes}")
        if cfg.get('beta_debug', True):
            preview_srt(merged_output, "merged", n=3)
        merged_outputs.append(merged_output)
    else:
        for p in [1, 2]:
            if results[p]['success']:
                final_output = output_dir / f"{video.stem}.whisperjav.srt"
                shutil.copy2(results[p]['output'], final_output)
                merged_outputs.append(final_output)
                print(f"    ⚠ Using Pass {p} only (other pass failed)")
                if cfg.get('beta_debug', True):
                    preview_srt(final_output, f"final(pass{p})", n=3)
                break
        else:
            print(f"    ✗ Both passes failed!")

    all_results.append(results)

# Store context for Step 3
WHISPERJAV_NEW_SRTS = merged_outputs
WHISPERJAV_OUTPUT_DIR = output_dir 

status(f"\nCreated {len(merged_outputs)} merged subtitle file(s) in {output_dir}")

# ═══════════════════════════════════════════
# ADD CREDITS
# ═══════════════════════════════════════════
section("ADDING CREDITS")

if cfg['opening_credit'] or cfg['closing_credit'] and merged_outputs:
    credits_count = 0
    for srt_file in merged_outputs:
        try:
            content = srt_file.read_text(encoding='utf-8')
            if cfg['opening_credit']:
                content = f"0\n00:00:00,000 --> 00:00:00,500\n{cfg['opening_credit']}\n\n" + content
            if cfg['closing_credit']:
                content += f"\n9999\n23:59:58,000 --> 23:59:59,000\n{cfg['closing_credit']}\n"
            srt_file.write_text(content, encoding='utf-8')
            credits_count += 1
        except Exception as e:
            print(f"  Warning: Could not add credits to {srt_file.name}: {e}")
    status(f"Credits added to {credits_count} file(s)")
else:
    status("No credits configured or no files generated")

# ═══════════════════════════════════════════
# COMPLETE & PERSIST
# ═══════════════════════════════════════════
section("TWO-PASS TRANSCRIPTION COMPLETE")

mode_text = "parallel" if PARALLEL_MODE else "sequential"
msg = f"<b>✓ Done ({mode_text})!</b> {len(merged_outputs)} subtitle(s)."

# Create a handy ZIP bundle (all SRTs + per-pass logs + any JSON metadata)
zip_path = output_dir / f"{cfg['folder_name']}_transcriptions.zip"
try:
    import zipfile
    files_to_zip = []
    seen = set()
    if output_dir.exists():
        for pattern in ['*.srt', '*.log', '*.json']:
            for p in output_dir.rglob(pattern):
                if not p.is_file():
                    continue
                rp = str(p.resolve())
                if rp in seen:
                    continue
                seen.add(rp)
                files_to_zip.append(p)

    if files_to_zip:
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for p in sorted(files_to_zip, key=lambda x: str(x)):
                try:
                    arcname = p.relative_to(output_dir)
                except Exception:
                    arcname = p.name
                zipf.write(p, str(arcname))
        status(f"Bundled results into: {zip_path.name} ({len(files_to_zip)} files)")
    else:
        status("No output/log/metadata files found to zip", False)
except Exception as e:
    print(f"Failed to create zip: {e}")

if PLATFORM == 'kaggle':
    msg += f"<br>📂 <b>Kaggle Output:</b> Check the right sidebar 'Output' tab → <code>{cfg['folder_name']}/</code>.<br>⚠ <b>Important:</b> Download files manually NOW or use 'Save Version' to persist them!"
    if zip_path.exists():
        from IPython.display import FileLink
        # FileLink automatically serves relative to the working directory.
        # Check if output_dir is relative or absolute under /kaggle/working
        try:
             # Calculate path relative to CWD (/kaggle/working)
             rel_path = zip_path.relative_to(Path.cwd())
             display(FileLink(str(rel_path), result_html_prefix="<b>⬇ Click to download ZIP: </b>", result_html_suffix=" (Kaggle)"))
        except ValueError:
             # Fallback if path parsing fails
             display(FileLink(zip_path.name, result_html_prefix="<b>⬇ Click to download ZIP: </b>", result_html_suffix=" (Kaggle)"))

else:
    msg += f" saved to Google Drive/..."
    if zip_path.exists():
        try:
            from google.colab import files
            files.download(str(zip_path)) 
        except: pass

display(HTML(f'<div style="padding:8px 10px;background:#f0fdf4;border-radius:4px;border-left:2px solid #16a34a;font-size:10px">{msg}</div>'))

if cfg['subtitle_language'] == 'llm' and not cfg['api_key']:
    print("Note: AI translation skipped (no API key provided)")

if cfg['auto_disconnect']:
    print("\nAuto-disconnecting logic triggered...")
    time.sleep(10)
    if PLATFORM == 'colab':
        try:
            from google.colab import runtime
            runtime.unassign()
        except: pass
    elif PLATFORM == 'kaggle':
        # Kaggle specific exit/cleanup if needed? 
        # Usually script ending is enough for Save Version interaction
        pass


In [ ]:
#@title Step 3: AI Translation (if selected) { display-mode: "form" }
#@markdown Translate each subtitle file using AI (only runs if "English (AI translate)" selected)

import os, sys, subprocess, shlex, time
from pathlib import Path
from IPython.display import display, HTML

def status(msg, ok=True):
    icon = "✓" if ok else "✗"
    print(f"{icon} {msg}")

def section(title):
    print(f"\n{'─'*40}\n{title}\n{'─'*40}")

# Check prerequisites
if 'WHISPERJAV_CONFIG' not in dir():
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> Run Step 1 first</div>'))
    raise SystemExit()

if 'WHISPERJAV_NEW_SRTS' not in dir():
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> Run Step 2 first</div>'))
    raise SystemExit()

cfg = WHISPERJAV_CONFIG
new_srts = WHISPERJAV_NEW_SRTS
output_dir = WHISPERJAV_OUTPUT_DIR if 'WHISPERJAV_OUTPUT_DIR' in dir() else Path(os.getcwd())

# Re-detect platform just in case
if 'google.colab' in sys.modules: PLATFORM = 'colab'
elif os.path.exists('/kaggle'): PLATFORM = 'kaggle'
else: PLATFORM = 'local'

# Check if AI translation is needed
if cfg['subtitle_language'] != 'llm':
    display(HTML('<div style="padding:8px 10px;background:#f0f9ff;border-radius:4px;border-left:2px solid #3b82f6;font-size:10px"><b>ℹ Skipped:</b> AI translation not selected</div>'))
    raise SystemExit()

# ═══════════════════════════════════════════
# API KEY RESOLUTION (SECRETS MANAGER)
# ═══════════════════════════════════════════
def retrieve_key(service_name, config_key):
    """Try to get key from config, Kaggle secrets, or env."""
    if config_key and config_key.strip():
        return config_key.strip()
    
    # Try Kaggle Secrets
    if PLATFORM == 'kaggle':
        try:
            from kaggle_secrets import UserSecretsClient
            secrets = UserSecretsClient()
            candidates = [
                f"{service_name.upper()}_API_KEY",
                "API_KEY",
                "LLM_API_KEY",
                service_name.lower()
            ]
            for c in candidates:
                try:
                    k = secrets.get_secret(c)
                    if k: return k
                except: pass
        except: pass
        
    return ""

final_api_key = retrieve_key(cfg['translation_service'], cfg.get('api_key'))

if not final_api_key:
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> No API key provided (checked Config and Kaggle Secrets)</div>'))
    raise SystemExit()

if not new_srts:
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> No subtitle files to translate</div>'))
    raise SystemExit()

# Set up API key
env_map = {
    "deepseek": "DEEPSEEK_API_KEY",
    "openrouter": "OPENROUTER_API_KEY",
    "gemini": "GEMINI_API_KEY",
    "claude": "ANTHROPIC_API_KEY",
    "gpt": "OPENAI_API_KEY"
}
os.environ[env_map.get(cfg['translation_service'], "API_KEY")] = final_api_key

# ═══════════════════════════════════════════
# TRANSLATION LOOP
# ═══════════════════════════════════════════
section("AI TRANSLATION")
print(f"Provider: {cfg['translation_service']}")
print(f"Style: {cfg['_translation_style']}")
print(f"Files to translate: {len(new_srts)}\n")

translated_files = []
failed_files = []

for i, srt_file in enumerate(new_srts, 1):
    print(f"[{i}/{len(new_srts)}] Translating: {srt_file.name}")

    translate_cmd = [
        'whisperjav-translate',
        '-i', str(srt_file),
        '--provider', cfg['translation_service'],
        '-t', 'english',
        '--tone', cfg['translation_style'],
        '--stream'
    ]

    full_cmd = shlex.join(translate_cmd)

    try:
        process = subprocess.Popen(
            full_cmd,
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            bufsize=1,
            universal_newlines=True
        )

        for line in process.stderr:
            print(f"    {line}", end='')

        stdout_output, _ = process.communicate()

        if process.returncode == 0:
            output_path = stdout_output.strip()
            if output_path:
                translated_files.append(Path(output_path))
            status(f"Completed: {srt_file.name}")
        else:
            status(f"Failed: {srt_file.name}", False)
            failed_files.append(srt_file)

    except Exception as e:
        status(f"Error translating {srt_file.name}: {e}", False)
        failed_files.append(srt_file)

    print()

# ═══════════════════════════════════════════
# COMPLETE
# ═══════════════════════════════════════════
section("COMPLETE")

# Create ZIP of translations
if translated_files:
    trans_zip_path = output_dir / f"{cfg['folder_name']}_translations_{cfg['translation_service']}.zip"
    try:
        import zipfile
        with zipfile.ZipFile(trans_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for srt in translated_files:
                zipf.write(srt, srt.name)
        status(f"Bundled translations into: {trans_zip_path.name}")
        
        # Download triggers
        if PLATFORM == 'kaggle':
            try:
                from IPython.display import FileLink
                rel_path = trans_zip_path.relative_to(Path.cwd())
                display(FileLink(str(rel_path), result_html_prefix="<b>⬇ Click to download Translations ZIP: </b>", result_html_suffix=" (Kaggle)"))
            except:
                display(HTML(f'<b>⬇ <a href="{trans_zip_path.name}" target="_blank">Click here to download Translations ZIP</a></b> (Kaggle)'))

        elif PLATFORM == 'colab':
            try:
                from google.colab import files
                files.download(str(trans_zip_path))
            except: pass
            
    except Exception as e:
        print(f"Failed to create zip: {e}")

if failed_files:
    display(HTML(f'<div style="padding:8px 10px;background:#fef9c3;border-radius:4px;border-left:2px solid #ca8a04;font-size:10px"><b>⚠ Partially done!</b> {len(translated_files)}/{len(new_srts)} translated. {len(failed_files)} failed.</div>'))
else:
    if PLATFORM == 'kaggle':
        loc_msg = f"<br>📂 <b>files in 'Output' tab → <code>{cfg['folder_name']}/</code></b>. <br>⚠ <b>Download now</b> or they will be lost when session ends!"
    else:
        loc_msg = f"in {output_dir}"

    display(HTML(f'<div style="padding:8px 10px;background:#f0fdf4;border-radius:4px;border-left:2px solid #16a34a;font-size:10px"><b>✓ All done!</b> {len(new_srts)} Japanese + {len(translated_files)} English subtitle(s) {loc_msg}</div>'))

# Auto-disconnect
if cfg['auto_disconnect']:
    print("\nAuto-disconnecting in 10s...")
    time.sleep(10)
    if PLATFORM == 'colab':
        try:
            from google.colab import runtime
            runtime.unassign()
        except: pass
else:
    print("\nRemember to disconnect manually.")
